### Step 1: Import packages

In [ ]:
!pip install cartopy geopandas osmnx

In [15]:
import pandas as pd
import numpy as np
import netCDF4 as ncdf
import os
from datetime import date, timedelta
from math import pi
from tqdm.auto import tqdm

import matplotlib.pyplot as plt
import cartopy.crs as ccrs
from cartopy.mpl.gridliner import LONGITUDE_FORMATTER, LATITUDE_FORMATTER
import matplotlib.ticker as mticker

# geography
import geopandas as gpd
import osmnx as ox
import shapely
from shapely.geometry import Point
import sklearn.neighbors
dist = sklearn.neighbors.DistanceMetric.get_metric(
    'haversine'
)

# ignore warnings
import warnings
warnings.filterwarnings(
    'ignore'
)

In [2]:
from google.colab import drive
drive.mount('/content/gdrive')
data_path = '/content/gdrive/MyDrive/Classes/W210_capstone/data'

os.chdir(data_path)
os.listdir(data_path)

Mounted at /content/gdrive


['merra2_data', 'tl_2010_06_zcta510', 'tl_2010_06_zcta510.zip']

### Step 2: Define working directories

In [3]:
!ls merra2_data

2000  2002  2004  2006	2008  2010  2012  2014	2016  2018  output
2001  2003  2005  2007	2009  2011  2013  2015	2017  2019


In [4]:
# in_dir_zip_shapes = 'C:/Users/cilin/Research/CA_hospitals/Input/raw_data/census_geo/shapefiles_zcta/'
# in_dir = 'C:/Users/cilin/Research/CA_hospitals/Input/raw_data/winds/'
# in_health = 'C:/Users/cilin/Research/CA_hospitals/Input/final_data/health/'
# out_dir = 'C:/Users/cilin/Research/CA_hospitals/Input/final_data/winds/'

in_dir_zip_shapes = 'tl_2010_06_zcta510/'
in_dir = 'merra2_data/2002/'
in_health = 'C:/Users/cilin/Research/CA_hospitals/Input/final_data/health/'
out_dir = 'merra2_data/output/'

### Step 3: Define functions

``read_clean wind``

In [5]:
def read_clean_wind(verbose=True):
    ''''''
    # create empty df
    df = pd.DataFrame()

    for file in os.listdir(in_dir):
        if file.startswith('MERRA2'):
            if verbose:
                print(file.split('.')[2])

            ## read .nc file ##
            ###################
            data = ncdf.Dataset(
                in_dir + file, mode='r'
            )
            # print metadata
            #print(data)

            # grab vars of interest ##
            ##########################
            # longitude and latitude
            lons = data.variables['lon']
            lats = data.variables['lat']
            # 2-meter eastward wind m/s
            U2M = data.variables['U2M']
            # 2-meter northward wind m/s
            V2M = data.variables['V2M']

            # Replace vals #
            ################
            #\_FillValues with NaNs:
            U2M_nans = U2M[:]
            V2M_nans = V2M[:]
            _FillValueU2M = U2M._FillValue
            _FillValueV2M = V2M._FillValue
            U2M_nans[U2M_nans == _FillValueU2M] = np.nan
            V2M_nans[V2M_nans == _FillValueV2M] = np.nan

            # Add new vars #
            ################
            # calculate wind speed
            wspd = np.sqrt(U2M_nans**2+V2M_nans**2)

            # calculate wind direction in radians
            wdir = np.arctan2(V2M_nans, U2M_nans)
            
            # transform wind direction from radians to degrees
            #dir_to_degrees = np.mod(180+np.rad2deg(np.arctan2(V2M_nans, U2M_nans)), 360) # this computes "wind is blowing from"' meteorological convetion'
            wdir_to_degrees = np.mod(np.rad2deg(wdir), 360) # this computes "wind is blowing towards" 'oceonographic convention', see here: https://www.esri.com/arcgis-blog/products/product/analytics/displaying-speed-and-direction-symbology-from-u-and-v-vectors/
            
            
            ## transform to df ##
            #####################
            # create an empty df for wind speed and direction with size len(lats) x len(lons) 
            df_wdir = pd.DataFrame(index=lats[:], columns=lons[:])   
            df_wspd = pd.DataFrame(index=lats[:], columns=lons[:])
            
            # create an empty df for u and v components with size len(lats) x len(lons) 
            df_u = pd.DataFrame(index=lats[:], columns=lons[:])
            df_v = pd.DataFrame(index=lats[:], columns=lons[:])

            # populate each row in the empty df above with the wdir_meteo and wspd data and u and v components
            for idx, idx_val in enumerate(df_wdir.index):
                df_wdir.loc[idx_val, :] = wdir_to_degrees[0][idx]
                df_wspd.loc[idx_val, :] = wspd[0][idx]
                df_u.loc[idx_val, :] = U2M_nans[0][idx]
                df_v.loc[idx_val, :] = V2M_nans[0][idx]

            # add index (latitude) as column
            df_wdir.reset_index(
                drop=False,
                inplace=True
            )
            
            df_wdir.rename(
                columns={'index':'lat'},
                inplace=True
            )
            
            
            df_wspd.reset_index(
                drop=False,
                inplace=True
            )
            
            df_wspd.rename(
                columns={'index':'lat'},
                inplace=True
            )
            
            df_u.reset_index(
                drop=False,
                inplace=True
            )
            
            df_u.rename(
                columns={'index':'lat'},
                inplace=True
            )
            
            df_v.reset_index(
                drop=False,
                inplace=True
            )
            
            df_v.rename(
                columns={'index':'lat'},
                inplace=True
            )

            # transform from wide to long
            df_wdir = pd.melt(
                df_wdir, id_vars='lat',
                var_name='lon',
                value_vars=lons[:],
                value_name='wdir'
            )
            
            df_wspd = pd.melt(
                df_wspd,
                id_vars='lat',
                var_name='lon',
                value_vars=lons[:],
                value_name='wspd'
            )
            
            df_u = pd.melt(
                df_u, id_vars='lat',
                var_name='lon',
                value_vars=lons[:],
                value_name='u'
            )
            
            df_v = pd.melt(
                df_v, id_vars='lat',
                var_name='lon',
                value_vars=lons[:],
                value_name='v'
            )

            # concatenate df_wdir and df_wspd
            df_temp1 = df_wdir.merge(
                df_wspd,
                on=['lat', 'lon'],
                how='left'
            )
            
            # concatenate df_u and df_v
            df_temp2 = df_u.merge(
                df_v,
                on=['lat', 'lon'],
                how='left'
            )
            
            # concatenate df_temp1 and df_temp2
            df_temp = df_temp2.merge(
                df_temp1,
                on=['lat', 'lon'],
                how='left'
            )
            
            # add time stamp 
            df_temp['year_month'] = file.split('.')[2]

            df = pd.concat(
                [df_temp, df],
                axis=0
            )
   
    # keep values in min, max range of California geometry
    df = df[
        df.lon.ge(-125) & df.lon.le(-115) & df.lat.ge(32) & df.lat.le(42)
    ]
    
    # transform vars
    df['lat'] = df.lat.astype(float)
    df['lon'] = df.lon.astype(float)
    
    return df

``read census geom``

In [6]:
def read_census_geom():
    """ Read Census (lat, lon) coordinates for California zip-codes
    parameters:
    -----------
    None
    
    return:
    -------
    Df with osmnx_geom
    """
    ### Step 1 ### 
    ##############
    # Read the shapefiles for California's ZIP codes
    for file in os.listdir(in_dir_zip_shapes):
        if file.endswith('.shp'):
            gdf = gpd.read_file(in_dir_zip_shapes + file)

    # keep only cols of interest 
    # ('ZCTA5CE10' = 2010 Census ZIP codes,	'GEOID10' = 2010 Census Tract codes)
    gdf = gdf[
        ['ZCTA5CE10',
         'GEOID10',
         'geometry']
    ]
    
    
    ### Step 2 ###
    ###############
    # For each zip cpde extract polygon with (lat, lon) info

    zip_poly = pd.DataFrame()

    for idx, multipoly in enumerate(gdf.geometry):
        if isinstance(multipoly, shapely.geometry.polygon.Polygon):
            temp_df = pd.DataFrame(
                {
                    'lat': multipoly.exterior.coords.xy[1], 
                    'lon': multipoly.exterior.coords.xy[0],
                    'ZCTA10': gdf.loc[idx, 'ZCTA5CE10'],
                    'GEOID10': gdf.loc[idx, 'GEOID10']
                }
            )
            zip_poly = pd.concat(
                [zip_poly, temp_df],
                axis=0
            )

        if isinstance(multipoly, shapely.geometry.multipolygon.MultiPolygon):
            for poly in multipoly:
                temp_df = pd.DataFrame(
                    {
                        'lat': poly.exterior.coords.xy[1], 
                        'lon': poly.exterior.coords.xy[0],
                        'ZCTA10': gdf.loc[idx, 'ZCTA5CE10'],
                        'GEOID10': gdf.loc[idx, 'GEOID10']
                    }
                )
                zip_poly = pd.concat(
                    [zip_poly, temp_df],
                    axis=0
                )   
    

    # round (lat, lon) to 2 decimal points and add 0.005 to match the UW (lat, lon) values
    zip_poly['lat'] = zip_poly.lat.round(3)
    zip_poly['lon'] = zip_poly.lon.round(3)
    
    zip_poly.sort_values(
        by=['ZCTA10', 'lat', 'lon'],
        inplace=True
    )
    
    zip_poly.drop_duplicates(
        subset=['ZCTA10', 'lat', 'lon'],
        inplace=True
    )

    zip_poly.reset_index(
        drop=True,
        inplace=True
    )
    
    return zip_poly

``find zip (zcta) code for wind data``

In [7]:
def add_zcta_to_wind(df1, df2):
    '''
    params:
    -------
    df1: wind data
    df2: census geometry data
    
    return:
    -------
    '''
    
    # create labels
    df1['wind_lat_lon'] = [str(xy) for xy in zip(df1.lat, df1.lon)]
    df2['census_lat_lon'] = [str(xy) for xy in zip(df2.lat, df2.lon)]

    ## for each point in wind data find the nearest point in the census data ##
    ###############
    # keep only unique points in wind data
    df1_unique = df1.drop_duplicates(
        ['wind_lat_lon']
    )
    
    df2_unique = df2.drop_duplicates(
        ['census_lat_lon']
    )
    
    df1_unique.reset_index(
        drop=True,
        inplace=True
    )
    
    df2_unique.reset_index(
        drop=True,
        inplace=True
    )

    # transform to radians
    df1_unique['lat_r'] = np.radians(df1_unique.lat)
    df1_unique['lon_r'] = np.radians(df1_unique.lon)
    df2_unique['lat_r'] = np.radians(df2_unique.lat)
    df2_unique['lon_r'] = np.radians(df2_unique.lon)


    # compute pairwise distance (in miles)
    dist_matrix = (dist.pairwise(
        df2_unique[['lat_r', 'lon_r']],
        df1_unique[['lat_r', 'lon_r']]
    ))*3959

    # create a df from dist_matrix
    dist_matrix = pd.DataFrame(
        dist_matrix,
        index=df2_unique['census_lat_lon'],
        columns=df1_unique['wind_lat_lon']
    )
    
    # for each row (census_lat_lon point) extract the closest column (wind_lat_lon point) 
    closest_point = pd.DataFrame(
        dist_matrix.idxmin(axis=1),
        columns=['closest_wind_lat_lon']
    )
    
    closest_point.reset_index(
        drop=False,
        inplace=True
    )

    # merge with census data
    df2_unique = df2_unique.merge(
        closest_point,
        on='census_lat_lon',
        how='left'
    )
    
    # merge with census data 
    df2_unique = df2_unique.merge(
        df2[['census_lat_lon']],
        on=['census_lat_lon'],
        how='left'
    )
    
    # replicate df2_unique based on number of year_month entries in df1
    df2_unique = pd.concat(
        [df2_unique]*(df1.year_month.nunique()),
        axis=0
    )
    
    df2_unique.reset_index(
        drop=True,
        inplace=True
    )
    
    # add year_month column to df2_unique
    df2_unique['year_month'] = 0
    indeces = [n for n in range(1, df2_unique.shape[0]) if n%956926==0]

    year_month = np.sort(df1.year_month.unique())
    for idx, index in enumerate(indeces):
        if idx==0:
            df2_unique.iloc[0:indeces[idx], 8] = year_month[idx]
        else:
            df2_unique.iloc[indeces[idx-1]:indeces[idx], 8] = year_month[idx]
            
            
    # from df1 keep only cols of interest
    df1 = df1[
        ['year_month',
         'u',
         'v',
         'wdir',
         'wspd',
         'wind_lat_lon']
    ]
    
    # merge df2_unique with df1
    df2_unique = df2_unique.merge(
        df1,
        left_on=['year_month', 'closest_wind_lat_lon'],
        right_on=['year_month', 'wind_lat_lon'],
        how='left'
    )
    # keep only cols of interest
    df2_unique = df2_unique[
        ['lat',
         'lon',
         'ZCTA10',
         'u',
         'v',
         'wdir',
         'wspd',
         'year_month']
    ]
    
    df2_unique.dropna(
        inplace=True
    )
    
    df2_unique.reset_index(
        drop=True,
        inplace=True
    )
    
    df2_unique.drop_duplicates(
    ['year_month', 'ZCTA10'],
    inplace=True
    )

    df2_unique.reset_index(
        drop=True,
        inplace=True
    )
    
    return df2_unique

### Step 4: Clean data for each year

``wind``

In [8]:
os.path.join("merra2_data", "hi" + "there", "there")

'merra2_data/hithere/there'

In [9]:
def clean_weather(year): 
  in_dir = 'merra2_data/' + year + '/'
  df = read_clean_wind(verbose=False)
  zip_poly = read_census_geom()
  df_final = add_zcta_to_wind(df, zip_poly)
  df_final.to_parquet(os.path.join('merra2_data/output', 'wind_' + year + '.parquet'))

In [17]:
for i in tqdm(range(2003, 2020)):
  clean_weather(str(i))

  0%|          | 0/17 [00:00<?, ?it/s]